In [19]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

In [20]:
def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform)
    val_dataset   = datasets.ImageFolder(os.path.join(data_dir, "val"), transform)
    test_dataset  = datasets.ImageFolder(os.path.join(data_dir, "test"), transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, len(train_dataset.classes)

In [21]:
#built-in AlexNet

def built_in_AlexNet():
    data_dir = r"D:\STAI_PROJECT\github\Dangerous-Farm-Insects-Classification-main\data\processed\farm_insects\splits" #changable
    num_epochs = 10

    train_loader, val_loader, test_loader, num_classes = get_dataloaders(data_dir)

    model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
    model.classifier[6] = nn.Linear(4096, num_classes)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    best_val_acc = 0.0

    for epoch in range(num_epochs):
        #train
        model.train()
        running_loss, correct, total = 0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        train_loss = running_loss / len(train_loader)

        #validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = outputs.max(1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        val_loss = val_loss / len(val_loader)

        print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
        print("-" * 60)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "alexnet_best_val.pth")

    print("Training complete. Best model saved as alexnet_best_val.pth")

    #test
    print("\nEvaluating on Test Set...")
    model.load_state_dict(torch.load("alexnet_best_val.pth"))
    model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="[Test]"):
            outputs = model(images)
            _, preds = outputs.max(1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = 100 * test_correct / test_total
    print(f"Final Test Accuracy: {test_acc:.2f}%")

if __name__ == "__main__":
    built_in_AlexNet()

Epoch 1/10 [Train]:  28%|██▊       | 9/32 [00:16<00:41,  1.82s/it]c:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1/10 [Val]: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]



Epoch 1/10 Summary:
  Train Loss: 2.0870 | Train Acc: 36.08%
  Val Loss:   1.5083 | Val Acc:   53.36%
------------------------------------------------------------


Epoch 2/10 [Val]: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]



Epoch 2/10 Summary:
  Train Loss: 0.9865 | Train Acc: 69.77%
  Val Loss:   1.2383 | Val Acc:   60.87%
------------------------------------------------------------


Epoch 3/10 [Val]: 100%|██████████| 8/8 [00:07<00:00,  1.08it/s]


...


Epoch 9/10 Summary:
  Train Loss: 0.0755 | Train Acc: 97.62%
  Val Loss:   1.3965 | Val Acc:   63.64%
------------------------------------------------------------


Epoch 10/10 [Val]: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]



Epoch 10/10 Summary:
  Train Loss: 0.0607 | Train Acc: 98.41%
  Val Loss:   1.4516 | Val Acc:   64.43%
------------------------------------------------------------
Training complete. Best model saved as alexnet_best_val.pth

Evaluating on Test Set...


[Test]: 100%|██████████| 10/10 [00:11<00:00,  1.20s/it]

Final Test Accuracy: 67.09%


In [22]:
#from-scratch AlexNet

class AlexNetModel(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

def alexnet_from_scratch():
    data_dir = r"D:\STAI_PROJECT\github\Dangerous-Farm-Insects-Classification-main\data\processed\farm_insects\splits" #changable
    num_epochs = 10
    lr = 1e-4

    train_loader, val_loader, test_loader, num_classes = get_dataloaders(data_dir)

    model = AlexNetModel(num_classes=num_classes)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_acc = 0.0

    for epoch in range(num_epochs):
        #train
        model.train()
        running_loss, correct, total = 0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        train_loss = running_loss / len(train_loader)

        #validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = outputs.max(1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        val_loss = val_loss / len(val_loader)

        print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
        print("-" * 60)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "alexnet_scratch_best_val.pth")

    print("Training complete. Best model saved as alexnet_scratch_best_val.pth")

    #test
    print("\nEvaluating on Test Set...")
    model.load_state_dict(torch.load("alexnet_scratch_best_val.pth"))
    model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="[Test]"):
            outputs = model(images)
            _, preds = outputs.max(1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = 100 * test_correct / test_total
    print(f"Final Test Accuracy: {test_acc:.2f}%")

if __name__ == "__main__":
    alexnet_from_scratch()


Epoch 1/10 [Val]: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]



Epoch 1/10 Summary:
  Train Loss: 2.7104 | Train Acc: 6.94%
  Val Loss:   2.7030 | Val Acc:   7.11%
------------------------------------------------------------


Epoch 2/10 [Val]: 100%|██████████| 8/8 [00:07<00:00,  1.08it/s]



Epoch 2/10 Summary:
  Train Loss: 2.6744 | Train Acc: 10.31%
  Val Loss:   2.6687 | Val Acc:   9.88%
------------------------------------------------------------


Epoch 3/10 [Val]: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]


...


Epoch 9/10 Summary:
  Train Loss: 2.2871 | Train Acc: 23.89%
  Val Loss:   2.4325 | Val Acc:   20.95%
------------------------------------------------------------


Epoch 10/10 [Val]: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]



Epoch 10/10 Summary:
  Train Loss: 2.2228 | Train Acc: 26.26%
  Val Loss:   2.3738 | Val Acc:   21.74%
------------------------------------------------------------
Training complete. Best model saved as alexnet_scratch_best_val.pth

Evaluating on Test Set...


[Test]: 100%|██████████| 10/10 [00:09<00:00,  1.02it/s]

Final Test Accuracy: 23.10%


# Note:

The difference between the builtin version and from scratch version is that the builtin one was already initialized by already trained weights from ImageNet dataset while my version starts nearly with random wrights so it needs much more epochs and data to get better results 